<a href="https://colab.research.google.com/github/IVN-RIN/bio-med-BIT/blob/main/notebooks/BioBIT_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BioBIT Experiment For <u>Pre-Training</u>**

*Tommaso Buonocore, University of Pavia, 2022*<br>
*Claudio Crema, IRCCS Centro San Giovanni di Dio Fatebenefratelli, 2022*

*Last edited: 16/11/2022*

# Initialization

Short string describing the current run

In [ ]:
experiment_name = "Pretraining-MedvX"

## Imports

In [ ]:
%%capture
!pip uninstall -y tensorflow
!pip install transformers==4.23.1 install tokenizers install datasets

# Google Colab only
from google.colab import runtime, drive

#General
import torch
from itertools import chain
import pandas as pd
import numpy as np
import json
import os
from io import StringIO
import time

# HuggingFace Transformers
from transformers import BertTokenizerFast, BertForMaskedLM, AutoModelForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_from_disk, load_dataset

# Set device to GPU Cuda if available 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Session Info

In [ ]:
session_info = json.loads(os.popen("curl curl ipinfo.io").read())
if device=='cuda':
  gpu_info = pd.read_csv(StringIO(os.popen("nvidia-smi --query-gpu=gpu_name,memory.total --format=csv").read()),names=["name","memory"],header=0)
  session_info[f'gpus'] = [{'name': row["name"], 'memory': row["memory"]} for index, row in gpu_info.iterrows()] 
else: 
  session_info[f'gpus'] = []
session_info['time_start'] = time.strftime("%H:%M:%S", time.localtime())
session_info['experiment_name'] = experiment_name
session_info

Since we are dealing with large files, we are going to realy solely upon Google Drive for both inputs and outputs.

In [ ]:
drive.mount('/content/gdrive')

## Global Parameters

In [ ]:
original_model = "/content/gdrive/MyDrive/Colab Environments/biobert_models/bio-full"
#original_model = "dbmdz/bert-base-italian-xxl-cased"
output_model_path = "./med-orig"
#dataset arrow version
corpus_path = '/content/gdrive/MyDrive/Colab Environments/PRETRAIN/med-whole-clean'
#collection of txt corpora
corpus_paths_common_root = '/content/gdrive/MyDrive/Colab Environments/PRETRAIN/'
corpus_paths_txt = ['med-whole-clean/med-whole-clean.txt',
                    # 'medicitalia-dump/medicitalia_dump_blog.txt','medicitalia-dump/medicitalia_dump_minforma.txt','medicitalia-dump/medicitalia_dump_salute.txt',
                    # 'mypersonaltrainer-dump/dump_benessere.txt','mypersonaltrainer-dump/dump_biologia.txt','mypersonaltrainer-dump/dump_farmaci.txt','mypersonaltrainer-dump/dump_farmaci-malattie.txt',
                    # 'mypersonaltrainer-dump/dump_farmacologia.txt','mypersonaltrainer-dump/dump_fisiologia.txt','mypersonaltrainer-dump/dump_rimedi.txt','mypersonaltrainer-dump/dump_salute.txt',
                    # 'mypersonaltrainer-dump/dump_salute-benessere.txt','mypersonaltrainer-dump/dump_traumatologia-ortopedia.txt',
                    # 'mypersonaltrainer-dump/dump_rimedi.txt','mypersonaltrainer-dump/dump_salute.txt','dica33_dump.txt'
                    ]

# mlm task
mlm_probability = 0.15
pad_to_multiple_of_8 = True

# experience replay params
replay_freq = 100 
old_corpus_path = '/content/gdrive/MyDrive/Colab Environments/PRETRAIN/bio-small'

# mixout params
mixout_prob = 0.9

# optimizer params
learning_rate = 3e-5
layerwise_learning_rate_decay = 0.95
weight_decay = 0.01
adam_epsilon = 1e-6
use_bertadam = False

# scheduler params
epochs = 10
batch_size = 16
warmup_ratio = 0.02 #the number of warmup steps is 2% of the total training steps

# switch on/off CF mitigation techniques here
cf_is_on = {
    "mixout":True,
    "experience_replay":False,
    "llrd":True,
}

# Tokenizer


Get Model and Tokenizer from pretrained checkpoint

In [ ]:
#Use BertForPreTraining for ML and NSP and BertForMaskedLM to drop the NSP task as in Roberta
tokenizer = BertTokenizerFast.from_pretrained(original_model, max_len=512)
model = BertForMaskedLM.from_pretrained(original_model)

##3. Build the training Dataset

We'll build our dataset by applying our tokenizer to our text file.

In [ ]:
def prepare_dataset_for_mlm(raw_dataset,tokenizer, batched=True, load_from_cache_file = False):
  """
  We tokenize every text, then concatenate them together before splitting them in smaller parts.
  We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more efficient when it receives the `special_tokens_mask`.
  """
  column_names = raw_dataset["train"].column_names
  text_column_name = "text" if "text" in column_names else column_names[0]
  max_seq_length = tokenizer.model_max_length

  def tokenize_function(examples):
    return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

  tokenized_datasets = raw_dataset.map(
      tokenize_function,
      batched=batched,
      remove_columns=column_names,
      load_from_cache_file=load_from_cache_file,
      desc="Running tokenizer on every text in dataset",
  )

  def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= max_seq_length:
        total_length = (total_length // max_seq_length) * max_seq_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
        for k, t in concatenated_examples.items()
    }
    return result

  tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=batched,
    load_from_cache_file=load_from_cache_file,
    desc=f"Grouping texts in chunks of {max_seq_length}",
  )

  return tokenized_datasets["train"]

In [ ]:
#OPTION 1: dataset stored as arrow file
# raw_dataset = load_from_disk(corpus_path)
# train_dataset = prepare_dataset_for_mlm(raw_dataset,tokenizer)

#OPTION 2: dataset(s) stored as txt file(s)
raw_dataset = load_dataset("text", data_files={"train": [corpus_paths_common_root+s for s in corpus_paths_txt]})
train_dataset = prepare_dataset_for_mlm(raw_dataset,tokenizer)

# Catastrophic Forgetting Mitigation

## **Mixout**

Described in [Lee et al., 2020](https://arxiv.org/pdf/1909.11299.pdf), this technique regularizes learning to minimize the deviation from the target parameters.

<img src="https://github.com/bloodwass/mixout/raw/master/imgs/mixout.png"/>

Suppose that u and w are a target model parameter and a current model parameter, respectively. 


*   a) We first memorize the parameters of the vanilla network at u. 
*   b) In the dropout network, we randomly choose an input neuron to be dropped(a dotted neuron) with a probability of p. That is, all outgoing parameters from the dropped neuron are eliminated (dotted connections). 
*   c) In the mixout(u) network, the eliminated parameters in (b) are replaced by the corresponding parameters in (a). 

In other words, the mixout(u) network at w is the mixture of the vanilla network at u and the dropout network at w with a probability of p.

In [ ]:
##++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Created by: Cheolhyoung Lee
## Department of Mathematical Sciences, KAIST
## Email: cheolhyoung.lee@kaist.ac.kr
## Implementation of mixout from https://arxiv.org/abs/1909.11299
## "Mixout: Effective Regularization to Finetune Large-scale Pretrained Language Models"
##++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import math
import torch
import torch.nn.init as init
import torch.nn.functional as F
from torch.nn import Parameter
from torch.autograd.function import InplaceFunction
from typing import Optional
from collections import OrderedDict

class Mixout(InplaceFunction):
    # target: a weight tensor mixes with a input tensor
    # A forward method returns 
    # [(1 - Bernoulli(1 - p) mask) * target + (Bernoulli(1 - p) mask) * input - p * target]/(1 - p) 
    # where p is a mix probability of mixout.
    # A backward returns the gradient of the forward method.
    # Dropout is equivalent to the case of target=None. 
    # I modified the code of dropout in PyTorch. 
    @staticmethod
    def _make_noise(input:torch.Tensor) -> torch.Tensor:
        return input.new().resize_as_(input)

    @classmethod
    def forward(cls, 
                ctx, 
                input:torch.Tensor, 
                target:Optional["OrderedDict[str, torch.Tensor]"]=None, 
                p:float=0.0, 
                training:bool=False, 
                inplace:bool=False) -> torch.Tensor:

        if p < 0 or p > 1:
            raise ValueError(f"A mix probability of mixout has to be between 0 and 1,  but got {p}")

        if target is not None and input.size() != target.size():
            raise ValueError(f"A target tensor size must match with a input tensor size {input.size()}, but got {target.size()}")
        
        ctx.p = p    
        ctx.training = training
        
        if target is None:
            target = cls._make_noise(input)
            target.fill_(0)
        target = target.to(input.device)

        if inplace:
            ctx.mark_dirty(input)
            output = input
        else:
            output = input.clone()
        
        if ctx.p == 0 or not ctx.training:
            return output
        
        ctx.noise = cls._make_noise(input)
        if len(ctx.noise.size()) == 1:
            ctx.noise.bernoulli_(1 - ctx.p)
        else:
            ctx.noise[0].bernoulli_(1 - ctx.p)
            ctx.noise = ctx.noise[0].repeat(input.size()[0], *([1] * (len(input.size())-1)))
        ctx.noise.expand_as(input)
        
        if ctx.p == 1:
            output = target.clone()
        else:
            output = ((1 - ctx.noise) * target + ctx.noise * output - ctx.p * target) / (1 - ctx.p)
        
        return output


    @staticmethod
    def backward(ctx, grad_output:torch.Tensor) -> Optional[torch.Tensor]:
        if ctx.p > 0 and ctx.training:
            return grad_output * ctx.noise, None, None, None, None
        else:
            return grad_output, None, None, None, None


def mixout(input:torch.Tensor, 
           target:Optional["OrderedDict[str, torch.Tensor]"]=None, 
           p:float=0.0, 
           training:bool=False, 
           inplace:bool=False) -> torch.Tensor:

    return Mixout.apply(input, target, p, training, inplace)

class MixLinear(torch.nn.Module):
    __constants__ = ['bias', 'in_features', 'out_features']
    # If target is None, nn.Sequential(nn.Linear(m, n), MixLinear(m', n', p)) 
    # is equivalent to nn.Sequential(nn.Linear(m, n), nn.Dropout(p), nn.Linear(m', n')).
    # If you want to change a dropout layer to a mixout layer, 
    # you should replace nn.Linear right after nn.Dropout(p) with Mixout(p) 
    def __init__(self, 
                in_features:int, 
                out_features:int, 
                bias:bool=True, 
                target:Optional["OrderedDict[str, torch.Tensor]"]=None, 
                p:float=0.0) -> None:

        super(MixLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()
        self.target = target
        self.p = p

        if self.p < 0 or self.p > 1:
            raise ValueError(f"A mix probability of mixout has to be between 0 and 1,  but got {self.p}")
    
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
            
    def forward(self, input:torch.Tensor) -> torch.Tensor:
        return F.linear(input, mixout(self.weight, self.target, 
                                      self.p, self.training), self.bias)

    def extra_repr(self):
        type_ = 'drop' if self.target is None else 'mix'
        type_ += "out" 
        return f'{type_}={self.p}, in_features={self.in_features}, out_features={self.out_features}, bias={self.bias is not None}'

In [ ]:
from torch import nn
from copy import deepcopy

def replace_layer_for_mixout(module: nn.Module, mixout_prob: float) -> nn.Module:
        '''
        Replaces a single layer with the correct layer for use with Mixout.
        If module is nn.Dropout, replaces it with a Dropout where p = 0.
        If module is nn.Linear, replaces it with a MixLinear where p(mixout) = mixout_prob.
        In all other cases, returns the module unchanged.
        
            params:
                module (nn.Module)    : a module to replace for Mixout
                mixout_prob (float)   : the desired Mixout probability
            
            returns:
                module (nn.Module)    : the module set up for use with Mixout
        '''
        if isinstance(module, nn.Dropout):
            return nn.Dropout(0)
        elif isinstance(module, nn.Linear):
            target_state_dict   = deepcopy(module.state_dict())
            bias                = True if module.bias is not None else False
            new_module          = MixLinear(
                                    module.in_features,
                                    module.out_features,
                                    bias,
                                    target_state_dict['weight'],
                                    mixout_prob
                                )
            new_module.load_state_dict(target_state_dict)
            return new_module
        else:
            return module

def recursive_setattr(obj: 'any', attr: str, value: 'any') -> None:
    '''
    Recursively sets attributes for objects with children.
    
        params:
            obj (any)   : the object with children whose attribute is to be set
            attr (str)  : the (nested) attribute of the object, with levels indicated by '.'
                          for instance attr='attr1.attr2' sets the attr2 of obj.attr1 to
                          the passed value
            value (any) : what to set the attribute to
    '''
    attr = attr.split('.', 1)
    if len(attr) == 1:
        setattr(obj, attr[0], value)
    else:
        recursive_setattr(getattr(obj, attr[0]), attr[1], value)

In [ ]:
if cf_is_on["mixout"]:
  print('Adding mixout...')
  print(model)

  # use tuple to avoid OrderedDict warning
  for name, module in tuple(model.named_modules()):
      if name:
          recursive_setattr(model, name, replace_layer_for_mixout(module, mixout_prob=mixout_prob))

## **Experience Replay**

At a certain interval (1% replay rate) throughout the learning period, uniformly sample from stored examples in the memory and perform gradient updates of the encoder-decoder network based on the retrieved examples. It helps mitigate catestophic forgetting.

<img src="https://github.com/h3lio5/episodic-lifelong-learning/raw/master/images/train_infer_new.png"/>

In the [original paper](https://github.com/h3lio5/episodic-lifelong-learning/blob/master/main.py) this process is handled directly inside the training process. The "naive" version might be to inject a random sampled batch of old examples every N batches (according to the replay rate) directly into the training dataset.

It is not yet clear to me what are the implications of using the naive version instead of the original one. Maybe the repetition of the same examples over different epochs vs sampling different old examples every time?

trainer: https://github.com/huggingface/transformers/blob/v4.23.1/src/transformers/trainer.py#L209

The only way to implement experience replay in the Transformer's `Trainer` is to extend the `Trainer` class modifying the `inner_training_loop` method. In particular, we customize the inner loop to perform an additional forward+backward training step on a random sampled batch from old pretraining data every `replay_freq` steps. 

In [ ]:
# coding=utf-8
# Copyright 2020-present the HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The Trainer class, to easily train a 🤗 Transformers from scratch or finetune it on a new task.
"""

import contextlib
import functools
import glob
import inspect
import math
import os
import random
import re
import shutil
import sys
import time
import warnings
from collections.abc import Mapping
from pathlib import Path
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

from tqdm.auto import tqdm


# Integrations must be imported before ML frameworks:
from transformers.integrations import (  # isort: split
    default_hp_search_backend,
    get_reporting_integration_callbacks,
    hp_params,
    is_fairscale_available,
    is_optuna_available,
    is_ray_tune_available,
    is_sigopt_available,
    is_wandb_available,
    run_hp_search_optuna,
    run_hp_search_ray,
    run_hp_search_sigopt,
    run_hp_search_wandb,
)

import numpy as np
import torch
import torch.distributed as dist
from packaging import version
from torch import nn
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from huggingface_hub import Repository

from transformers import __version__
from transformers.configuration_utils import PretrainedConfig
from transformers.data.data_collator import DataCollator, DataCollatorWithPadding, default_data_collator
from transformers.debug_utils import DebugOption, DebugUnderflowOverflow
from transformers.deepspeed import deepspeed_init, is_deepspeed_zero3_enabled
from transformers.dependency_versions_check import dep_version_check
from transformers.modelcard import TrainingSummary
from transformers.modeling_utils import PreTrainedModel, load_sharded_checkpoint, unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES, MODEL_MAPPING_NAMES
from transformers.optimization import Adafactor, get_scheduler
from transformers.pytorch_utils import ALL_LAYERNORM_LAYERS, is_torch_greater_or_equal_than_1_10, is_torch_less_than_1_11
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.trainer_callback import (
    CallbackHandler,
    DefaultFlowCallback,
    PrinterCallback,
    ProgressCallback,
    TrainerCallback,
    TrainerControl,
    TrainerState,
)
from transformers.trainer_pt_utils import (
    DistributedLengthGroupedSampler,
    DistributedSamplerWithLoop,
    DistributedTensorGatherer,
    IterableDatasetShard,
    LabelSmoother,
    LengthGroupedSampler,
    SequentialDistributedSampler,
    ShardSampler,
    distributed_broadcast_scalars,
    distributed_concat,
    find_batch_size,
    get_module_class_from_name,
    get_parameter_names,
    nested_concat,
    nested_detach,
    nested_numpify,
    nested_truncate,
    nested_xla_mesh_reduce,
    reissue_pt_warnings,
)
from transformers.trainer_utils import (
    PREFIX_CHECKPOINT_DIR,
    BestRun,
    EvalLoopOutput,
    EvalPrediction,
    FSDPOption,
    HPSearchBackend,
    HubStrategy,
    IntervalStrategy,
    PredictionOutput,
    RemoveColumnsCollator,
    ShardedDDPOption,
    TrainerMemoryTracker,
    TrainOutput,
    default_compute_objective,
    default_hp_space,
    denumpify_detensorize,
    enable_full_determinism,
    find_executable_batch_size,
    get_last_checkpoint,
    has_length,
    number_of_arguments,
    seed_worker,
    set_seed,
    speed_metrics,
)
from transformers.training_args import OptimizerNames, ParallelMode, TrainingArguments
from transformers.utils import (
    CONFIG_NAME,
    WEIGHTS_INDEX_NAME,
    WEIGHTS_NAME,
    find_labels,
    get_full_repo_name,
    is_apex_available,
    is_datasets_available,
    is_in_notebook,
    is_ipex_available,
    is_sagemaker_dp_enabled,
    is_sagemaker_mp_enabled,
    is_torch_tensorrt_fx_available,
    is_torch_tpu_available,
    is_torchdynamo_available,
    logging,
)
from transformers.utils.generic import ContextManagers


_is_native_cpu_amp_available = is_torch_greater_or_equal_than_1_10

DEFAULT_CALLBACKS = [DefaultFlowCallback]
DEFAULT_PROGRESS_CALLBACK = ProgressCallback

if is_in_notebook():
    from transformers.utils.notebook import NotebookProgressCallback

    DEFAULT_PROGRESS_CALLBACK = NotebookProgressCallback

if is_apex_available():
    from apex import amp

if is_datasets_available():
    import datasets

if is_torch_tpu_available(check_device=False):
    import torch_xla.core.xla_model as xm
    import torch_xla.debug.metrics as met
    import torch_xla.distributed.parallel_loader as pl

if is_fairscale_available():
    dep_version_check("fairscale")
    import fairscale
    from fairscale.nn.data_parallel import FullyShardedDataParallel as FullyShardedDDP
    from fairscale.nn.data_parallel import ShardedDataParallel as ShardedDDP
    from fairscale.nn.wrap import auto_wrap
    from fairscale.optim import OSS
    from fairscale.optim.grad_scaler import ShardedGradScaler


if is_sagemaker_mp_enabled():
    import smdistributed.modelparallel.torch as smp
    from smdistributed.modelparallel import __version__ as SMP_VERSION

    IS_SAGEMAKER_MP_POST_1_10 = version.parse(SMP_VERSION) >= version.parse("1.10")

    from transformers.trainer_pt_utils import smp_forward_backward, smp_forward_only, smp_gather, smp_nested_concat
else:
    IS_SAGEMAKER_MP_POST_1_10 = False


if TYPE_CHECKING:
    import optuna

logger = logging.get_logger(__name__)


# Name of the files used for checkpointing
TRAINING_ARGS_NAME = "training_args.bin"
TRAINER_STATE_NAME = "trainer_state.json"
OPTIMIZER_NAME = "optimizer.pt"
SCHEDULER_NAME = "scheduler.pt"
SCALER_NAME = "scaler.pt"

####From Trainer source code, Transformers v4.23.1
class ExperienceReplayTrainer(Trainer):

    ##################################################################
    #######  EXPERIENCE REPLAY MODIFICATION STARTS HERE  #############
    ##################################################################

    def __init__(self, old_dataset, replay_freq, *args, **kwargs):  #we pass the old dataset and replay freq to init
        self.old_dataset = old_dataset
        self.replay_freq = replay_freq
        super().__init__(*args, **kwargs)

    def get_old_dataloader(self) -> DataLoader:
        """
        dataloader for experience replay. NOTE: shuffle=True
        """
        if self.old_dataset is None:
            raise ValueError("Trainer: experience replay requires a old_dataset.")

        old_dataset = self.old_dataset
        data_collator = self.data_collator
        if is_datasets_available() and isinstance(old_dataset, datasets.Dataset):
            old_dataset = self._remove_unused_columns(old_dataset, description="experience replay")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="experience replay")

        if isinstance(old_dataset, torch.utils.data.IterableDataset):
            if self.args.world_size > 1:
                old_dataset = IterableDatasetShard(
                    old_dataset,
                    batch_size=self._train_batch_size,
                    drop_last=self.args.dataloader_drop_last,
                    num_processes=self.args.world_size,
                    process_index=self.args.process_index,
                )

            return DataLoader(
                old_dataset,
                batch_size=self.args.per_device_train_batch_size,
                shuffle=True,
                collate_fn=data_collator,
                num_workers=self.args.dataloader_num_workers,
                pin_memory=self.args.dataloader_pin_memory,
            )

        return DataLoader(
            old_dataset,
            batch_size=self._train_batch_size,
            shuffle=True,
            collate_fn=data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
            worker_init_fn=seed_worker,
        )

    ##################################################################
    #######  EXPERIENCE REPLAY MODIFICATION ENDS HERE  ###############
    ##################################################################

    def _inner_training_loop(
        self, batch_size=None, args=None, resume_from_checkpoint=None, trial=None, ignore_keys_for_eval=None
    ):
        self._train_batch_size = batch_size
        # Data loader and number of training steps
        train_dataloader = self.get_train_dataloader()

        ##################################################################
        #######  EXPERIENCE REPLAY MODIFICATION STARTS HERE  #############
        ##################################################################       

        old_dataloader = self.get_old_dataloader()

        ##################################################################
        #######  EXPERIENCE REPLAY MODIFICATION ENDS HERE  ###############
        ################################################################## 

        # Setting up training control variables:
        # number of training epochs: num_train_epochs
        # number of training steps per epoch: num_update_steps_per_epoch
        # total number of training steps to execute: max_steps
        total_train_batch_size = args.train_batch_size * args.gradient_accumulation_steps * args.world_size

        len_dataloader = None
        if has_length(train_dataloader):
            len_dataloader = len(train_dataloader)
            num_update_steps_per_epoch = len_dataloader // args.gradient_accumulation_steps
            num_update_steps_per_epoch = max(num_update_steps_per_epoch, 1)
            num_examples = self.num_examples(train_dataloader)
            if args.max_steps > 0:
                max_steps = args.max_steps
                num_train_epochs = args.max_steps // num_update_steps_per_epoch + int(
                    args.max_steps % num_update_steps_per_epoch > 0
                )
                # May be slightly incorrect if the last batch in the training dataloader has a smaller size but it's
                # the best we can do.
                num_train_samples = args.max_steps * total_train_batch_size
            else:
                max_steps = math.ceil(args.num_train_epochs * num_update_steps_per_epoch)
                num_train_epochs = math.ceil(args.num_train_epochs)
                num_train_samples = self.num_examples(train_dataloader) * args.num_train_epochs
        elif args.max_steps > 0:  # Rely on max_steps when dataloader does not have a working size
            max_steps = args.max_steps
            # Setting a very large number of epochs so we go as many times as necessary over the iterator.
            num_train_epochs = sys.maxsize
            num_update_steps_per_epoch = max_steps
            num_examples = total_train_batch_size * args.max_steps
            num_train_samples = args.max_steps * total_train_batch_size
        else:
            raise ValueError(
                "args.max_steps must be set to a positive value if dataloader does not have a length, was"
                f" {args.max_steps}"
            )

        if DebugOption.UNDERFLOW_OVERFLOW in self.args.debug:
            if self.args.n_gpu > 1:
                # nn.DataParallel(model) replicates the model, creating new variables and module
                # references registered here no longer work on other gpus, breaking the module
                raise ValueError(
                    "Currently --debug underflow_overflow is not supported under DP. Please use DDP"
                    " (torch.distributed.launch)."
                )
            else:
                debug_overflow = DebugUnderflowOverflow(self.model)  # noqa

        delay_optimizer_creation = (
            self.sharded_ddp is not None
            and self.sharded_ddp != ShardedDDPOption.SIMPLE
            or is_sagemaker_mp_enabled()
            or self.fsdp is not None
        )
        if args.deepspeed:
            deepspeed_engine, optimizer, lr_scheduler = deepspeed_init(
                self, num_training_steps=max_steps, resume_from_checkpoint=resume_from_checkpoint
            )
            self.model = deepspeed_engine.module
            self.model_wrapped = deepspeed_engine
            self.deepspeed = deepspeed_engine
            self.optimizer = optimizer
            self.lr_scheduler = lr_scheduler
        elif not delay_optimizer_creation:
            self.create_optimizer_and_scheduler(num_training_steps=max_steps)

        self.state = TrainerState()
        self.state.is_hyper_param_search = trial is not None

        # Activate gradient checkpointing if needed
        if args.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        model = self._wrap_model(self.model_wrapped)

        if is_sagemaker_mp_enabled() and resume_from_checkpoint is not None:
            self._load_from_checkpoint(resume_from_checkpoint, model)

        # for the rest of this function `model` is the outside model, whether it was wrapped or not
        if model is not self.model:
            self.model_wrapped = model

        if delay_optimizer_creation:
            self.create_optimizer_and_scheduler(num_training_steps=max_steps)

        # Check if saved optimizer or scheduler states exist
        self._load_optimizer_and_scheduler(resume_from_checkpoint)

        # important: at this point:
        # self.model         is the Transformers Model
        # self.model_wrapped is DDP(Transformers Model), Deepspeed(Transformers Model), etc.

        # Train!
        logger.info("***** Running training *****")
        logger.info(f"  Num examples = {num_examples}")
        logger.info(f"  Num Epochs = {num_train_epochs}")
        logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
        logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_train_batch_size}")
        logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
        logger.info(f"  Total optimization steps = {max_steps}")

        self.state.epoch = 0
        start_time = time.time()
        epochs_trained = 0
        steps_trained_in_current_epoch = 0
        steps_trained_progress_bar = None

        # Check if continuing training from a checkpoint
        if resume_from_checkpoint is not None and os.path.isfile(
            os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME)
        ):
            self.state = TrainerState.load_from_json(os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME))
            epochs_trained = self.state.global_step // num_update_steps_per_epoch
            if not args.ignore_data_skip:
                steps_trained_in_current_epoch = self.state.global_step % (num_update_steps_per_epoch)
                steps_trained_in_current_epoch *= args.gradient_accumulation_steps
            else:
                steps_trained_in_current_epoch = 0

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info(f"  Continuing training from epoch {epochs_trained}")
            logger.info(f"  Continuing training from global step {self.state.global_step}")
            if not args.ignore_data_skip:
                logger.info(
                    f"  Will skip the first {epochs_trained} epochs then the first {steps_trained_in_current_epoch} "
                    "batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` "
                    "flag to your launch command, but you will resume the training on data already seen by your model."
                )
                if self.is_local_process_zero() and not args.disable_tqdm:
                    steps_trained_progress_bar = tqdm(total=steps_trained_in_current_epoch)
                    steps_trained_progress_bar.set_description("Skipping the first batches")

        # Update the references
        self.callback_handler.model = self.model
        self.callback_handler.optimizer = self.optimizer
        self.callback_handler.lr_scheduler = self.lr_scheduler
        self.callback_handler.train_dataloader = train_dataloader
        self.state.trial_name = self.hp_name(trial) if self.hp_name is not None else None
        if trial is not None:
            assignments = trial.assignments if self.hp_search_backend == HPSearchBackend.SIGOPT else trial
            self.state.trial_params = hp_params(assignments)
        else:
            self.state.trial_params = None
        # This should be the same if the state has been saved but in case the training arguments changed, it's safer
        # to set this after the load.
        self.state.max_steps = max_steps
        self.state.num_train_epochs = num_train_epochs
        self.state.is_local_process_zero = self.is_local_process_zero()
        self.state.is_world_process_zero = self.is_world_process_zero()

        # tr_loss is a tensor to avoid synchronization of TPUs through .item()
        tr_loss = torch.tensor(0.0).to(args.device)
        # _total_loss_scalar is updated everytime .item() has to be called on tr_loss and stores the sum of all losses
        self._total_loss_scalar = 0.0
        self._globalstep_last_logged = self.state.global_step
        model.zero_grad()

        self.control = self.callback_handler.on_train_begin(args, self.state, self.control)

        # Skip the first epochs_trained epochs to get the random state of the dataloader at the right point.
        if not args.ignore_data_skip:
            for epoch in range(epochs_trained):
                is_random_sampler = hasattr(train_dataloader, "sampler") and isinstance(
                    train_dataloader.sampler, RandomSampler
                )
                if is_torch_less_than_1_11 or not is_random_sampler:
                    # We just need to begin an iteration to create the randomization of the sampler.
                    # That was before PyTorch 1.11 however...
                    for _ in train_dataloader:
                        break
                else:
                    # Otherwise we need to call the whooooole sampler cause there is some random operation added
                    # AT THE VERY END!
                    _ = list(train_dataloader.sampler)

        for epoch in range(epochs_trained, num_train_epochs):
            if isinstance(train_dataloader, DataLoader) and isinstance(train_dataloader.sampler, DistributedSampler):
                train_dataloader.sampler.set_epoch(epoch)
            elif hasattr(train_dataloader, "dataset") and isinstance(train_dataloader.dataset, IterableDatasetShard):
                train_dataloader.dataset.set_epoch(epoch)

            if is_torch_tpu_available():
                parallel_loader = pl.ParallelLoader(train_dataloader, [args.device]).per_device_loader(args.device)
                epoch_iterator = parallel_loader
            else:
                epoch_iterator = train_dataloader

            # Reset the past mems state at the beginning of each epoch if necessary.
            if args.past_index >= 0:
                self._past = None

            steps_in_epoch = (
                len(epoch_iterator)
                if len_dataloader is not None
                else args.max_steps * args.gradient_accumulation_steps
            )
            self.control = self.callback_handler.on_epoch_begin(args, self.state, self.control)

            if epoch == epochs_trained and resume_from_checkpoint is not None and steps_trained_in_current_epoch == 0:
                self._load_rng_state(resume_from_checkpoint)

            step = -1
            for step, inputs in enumerate(epoch_iterator):

                ##################################################################
                #######  EXPERIENCE REPLAY MODIFICATION STARTS HERE  #############
                ##################################################################

                #### Note: there might be a OOM problem because we are loading two
                #### batches in the same step: inputs and old_inputs. If it does
                #### not work, try to move 'inputs' back to cpu and then reload it
                #### to the gpu when you're done with 'old_inputs'.

                #option 1 (used in the original paper)
                #inputs_cp = copy.deepcopy(inputs)
                #del inputs

                #option 2 (experimental if option 1 won't work)
                #for input_tensor in inputs:
                #     input_tensor.to("cpu")
                #//experience replay code chunk here
                #for input_tensor in inputs:
                #     input_tensor.to(self.args.device)

                if (step+1) % replay_freq == 0:           #every replay_freq steps

                    #sample a new batch from the old data
                    old_inputs = next(iter(old_dataloader))    # <<------ load next random batch
                    self.optimizer.zero_grad()                 # clear out the gradients (by default they accumulate)
                    tr_loss_step = self.training_step(model, old_inputs)
                    tr_loss += tr_loss_step
                    del old_inputs

                ##################################################################
                #######  EXPERIENCE REPLAY MODIFICATION ENDS HERE  ###############
                ##################################################################

                # Skip past any already trained steps if resuming training
                if steps_trained_in_current_epoch > 0:
                    steps_trained_in_current_epoch -= 1
                    if steps_trained_progress_bar is not None:
                        steps_trained_progress_bar.update(1)
                    if steps_trained_in_current_epoch == 0:
                        self._load_rng_state(resume_from_checkpoint)
                    continue
                elif steps_trained_progress_bar is not None:
                    steps_trained_progress_bar.close()
                    steps_trained_progress_bar = None

                if step % args.gradient_accumulation_steps == 0:
                    self.control = self.callback_handler.on_step_begin(args, self.state, self.control)

                if (
                    ((step + 1) % args.gradient_accumulation_steps != 0)
                    and args.local_rank != -1
                    and args._no_sync_in_gradient_accumulation
                ):
                    # Avoid unnecessary DDP synchronization since there will be no backward pass on this example.
                    with model.no_sync():
                        tr_loss_step = self.training_step(model, inputs)
                else:
                    tr_loss_step = self.training_step(model, inputs)

                if (
                    args.logging_nan_inf_filter
                    and not is_torch_tpu_available()
                    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
                ):
                    # if loss is nan or inf simply add the average of previous logged losses
                    tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_last_logged)
                else:
                    tr_loss += tr_loss_step

                self.current_flos += float(self.floating_point_ops(inputs))

                # Optimizer step for deepspeed must be called on every step regardless of the value of gradient_accumulation_steps
                if self.deepspeed:
                    self.deepspeed.step()

                if (step + 1) % args.gradient_accumulation_steps == 0 or (
                    # last step in epoch but step is always smaller than gradient_accumulation_steps
                    steps_in_epoch <= args.gradient_accumulation_steps
                    and (step + 1) == steps_in_epoch
                ):
                    # Gradient clipping
                    if args.max_grad_norm is not None and args.max_grad_norm > 0 and not self.deepspeed:
                        # deepspeed does its own clipping

                        if self.do_grad_scaling:
                            # Reduce gradients first for XLA
                            if is_torch_tpu_available():
                                gradients = xm._fetch_gradients(self.optimizer)
                                xm.all_reduce("sum", gradients, scale=1.0 / xm.xrt_world_size())
                            # AMP: gradients need unscaling
                            self.scaler.unscale_(self.optimizer)

                        if is_sagemaker_mp_enabled() and args.fp16:
                            self.optimizer.clip_master_grads(args.max_grad_norm)
                        elif hasattr(self.optimizer, "clip_grad_norm"):
                            # Some optimizers (like the sharded optimizer) have a specific way to do gradient clipping
                            self.optimizer.clip_grad_norm(args.max_grad_norm)
                        elif hasattr(model, "clip_grad_norm_"):
                            # Some models (like FullyShardedDDP) have a specific way to do gradient clipping
                            model.clip_grad_norm_(args.max_grad_norm)
                        else:
                            # Revert to normal clipping otherwise, handling Apex or full precision
                            nn.utils.clip_grad_norm_(
                                amp.master_params(self.optimizer) if self.use_apex else model.parameters(),
                                args.max_grad_norm,
                            )

                    # Optimizer step
                    optimizer_was_run = True
                    if self.deepspeed:
                        pass  # called outside the loop
                    elif is_torch_tpu_available():
                        if self.do_grad_scaling:
                            self.scaler.step(self.optimizer)
                            self.scaler.update()
                        else:
                            xm.optimizer_step(self.optimizer)
                    elif self.do_grad_scaling:
                        scale_before = self.scaler.get_scale()
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        scale_after = self.scaler.get_scale()
                        optimizer_was_run = scale_before <= scale_after
                    else:
                        self.optimizer.step()

                    if optimizer_was_run and not self.deepspeed:
                        self.lr_scheduler.step()

                    model.zero_grad()
                    self.state.global_step += 1
                    self.state.epoch = epoch + (step + 1) / steps_in_epoch
                    self.control = self.callback_handler.on_step_end(args, self.state, self.control)

                    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
                else:
                    self.control = self.callback_handler.on_substep_end(args, self.state, self.control)

                if self.control.should_epoch_stop or self.control.should_training_stop:
                    break
            if step < 0:
                logger.warning(
                    "There seems to be not a single sample in your epoch_iterator, stopping training at step"
                    f" {self.state.global_step}! This is expected if you're using an IterableDataset and set"
                    f" num_steps ({max_steps}) higher than the number of available samples."
                )
                self.control.should_training_stop = True

            self.control = self.callback_handler.on_epoch_end(args, self.state, self.control)
            self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)

            if DebugOption.TPU_METRICS_DEBUG in self.args.debug:
                if is_torch_tpu_available():
                    # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
                    xm.master_print(met.metrics_report())
                else:
                    logger.warning(
                        "You enabled PyTorch/XLA debug metrics but you don't have a TPU "
                        "configured. Check your training configuration if this is unexpected."
                    )
            if self.control.should_training_stop:
                break

        if args.past_index and hasattr(self, "_past"):
            # Clean the state at the end of training
            delattr(self, "_past")

        logger.info("\n\nTraining completed. Do not forget to share your model on huggingface.co/models =)\n\n")
        if args.load_best_model_at_end and self.state.best_model_checkpoint is not None:
            # Wait for everyone to get here so we are sur the model has been saved by process 0.
            if is_torch_tpu_available():
                xm.rendezvous("load_best_model_at_end")
            elif args.local_rank != -1:
                dist.barrier()
            elif is_sagemaker_mp_enabled():
                smp.barrier()

            self._load_best_model()

        # add remaining tr_loss
        self._total_loss_scalar += tr_loss.item()
        train_loss = self._total_loss_scalar / self.state.global_step

        metrics = speed_metrics("train", start_time, num_samples=num_train_samples, num_steps=self.state.max_steps)
        self.store_flos()
        metrics["total_flos"] = self.state.total_flos
        metrics["train_loss"] = train_loss

        self.is_in_train = False

        self._memory_tracker.stop_and_update_metrics(metrics)

        self.log(metrics)

        self.control = self.callback_handler.on_train_end(args, self.state, self.control)

        return TrainOutput(self.state.global_step, train_loss, metrics)

## **Layer-wise Learning Rate Decay (LLRD)**

LLRD is a method that applies higher learning rates for top layers and lower learning rates for bottom layers. This is accomplished by setting the learning rate of the top layer and using a multiplicative decay rate to decrease the learning rate layer-by-layer from top to bottom.

The goal is to modify the lower layers that encode more general information less than the top layers that are more specific to the pre-training task. This method is adopted in fine-tuning several recent pre-trained models, including XLNet and ELECTRA, but not BERT.

In [ ]:
from transformers import (
    AdamW, 
    AutoConfig, 
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup
)
from transformers import logging
logging.set_verbosity_warning()
logging.set_verbosity_error()

config = AutoConfig.from_pretrained(original_model)

In [ ]:
def get_optimizer_grouped_parameters(model, model_type, learning_rate, weight_decay, layerwise_learning_rate_decay):
    no_decay = ["bias", "LayerNorm.weight"]
    # initialize lr for task specific layer
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if "classifier" in n or "pooler" in n],
            "weight_decay": 0.0,
            "lr": learning_rate,
        },
    ]
    # initialize lrs for every layer
    num_layers = model.config.num_hidden_layers
    layers = [getattr(model, model_type).embeddings] + list(getattr(model, model_type).encoder.layer)
    layers.reverse()
    lr = learning_rate
    for layer in layers:
        lr *= layerwise_learning_rate_decay
        optimizer_grouped_parameters += [
            {
                "params": [p for n, p in layer.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": weight_decay,
                "lr": lr,
            },
            {
                "params": [p for n, p in layer.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
                "lr": lr,
            },
        ]
    return optimizer_grouped_parameters

In [ ]:
grouped_optimizer_params = get_optimizer_grouped_parameters(model=model, 
                                                            model_type='bert', 
                                                            learning_rate=learning_rate, 
                                                            weight_decay=weight_decay, 
                                                            layerwise_learning_rate_decay=layerwise_learning_rate_decay)
optimizer = AdamW(params=grouped_optimizer_params, 
                  lr=learning_rate, 
                  eps=adam_epsilon, 
                  correct_bias=not use_bertadam)

tot_steps = math.ceil(len(train_dataset)/batch_size)*epochs
num_warmup_steps = int(warmup_ratio*tot_steps)

scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=num_warmup_steps,
                                            num_training_steps=epochs)

if cf_is_on["llrd"]:
  optimizers = (optimizer,scheduler)
else:
  optimizers = (None, None)

#Training 

We need to define a data_collator. This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=mlm_probability,
    pad_to_multiple_of=8 if pad_to_multiple_of_8 else None,
)

## Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir = output_model_path,
    save_strategy = "epoch",
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size = batch_size,
    save_total_limit=2,
    prediction_loss_only=True,
    disable_tqdm=False
)

if cf_is_on["experience_replay"]:
  raw_datasets_old = load_from_disk(old_corpus_path)
  old_dataset = prepare_dataset_for_mlm(raw_datasets_old,tokenizer)  #N.B. we are assuming that the old and new model share the same tokenizer. If not, use old tokenizer
  trainer = ExperienceReplayTrainer(
      model = model,
      args = training_args,
      data_collator = data_collator,
      train_dataset = train_dataset,
      old_dataset = old_dataset,
      replay_freq = replay_freq,
      optimizers= optimizers
  )
else:
  trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers= optimizers
  )

## Training Loop

In [ ]:
trainer.train()

## Save model

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/Colab Environments/biobert_models/newmodel") # remember to rename newmodel after saving to leave this path empty for the next iteration
tokenizer.save_pretrained("/content/gdrive/MyDrive/Colab Environments/biobert_models/newmodel")

Finalize session info and download

In [ ]:
session_info['original_checkpoint'] = [original_model]
session_info['cf'] = cf_is_on
session_info['training_arguments'] = training_args
session_info['time_end'] = time.strftime("%H:%M:%S", time.localtime())

with open(f'/content/session_info.json', "w") as outfile:
    outfile.write(json.dumps(session_info, indent=4))
files.download(f'/content/session_info.json')

If you don't want to check the model, unassign the runtime here to save compute units

In [ ]:
runtime.unassign()

# (Optional) Model check

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model = "/content/gdrive/MyDrive/Colab Environments/biobert_models/newmodel",
    tokenizer = original_model
)

In [ ]:
# The sun <mask>.

fill_mask("Un tumore cerebrale consiste nella crescita anomale di cellule nei tessuti del [MASK], che può essere benigna o maligna")

In [ ]:
runtime.unassign()